## 2. Initial Data Transformation

## Task description
Join the equivalent of the contents of the file city-hex-polygons-8.geojson to the service request dataset, such that each service request is assigned to a single H3 resolution level 8 hexagon. Use the sr_hex.csv.gz file to validate your work.

For any requests where the Latitude and Longitude fields are empty, set the index value to 0. Use your judgement to include any other appropriate validation.

Include logging that lets the executor know how many of the records failed to join, and include a join error threshold above which the script will error out. Please motivate why you have selected the error threshold that you have. Please also log the time taken to perform the operations described, and within reason, try to optimise latency and computational resources used.

In [ ]:
import sys
import logging
import time
import pandas as pd
import geopandas as gpd

Failure rate: the `FAILURE_THRESHOLD` is set to 5%. Without more content, this is a reasonable tolerance because:

- It allows for minor coordinate issues (e.g. bad GPS data, out-of-bounds requests).
- But it will catch systemic issues (e.g. wrong CRS or missing polygons).

In practice, a reasonable tolerance will be set taking into account the specific requirements of the request (e.g. the level of accuracy required and the 'cost' of an error).

In [ ]:
INPUT_PATH = "../data/sr.csv.gz"
HEX_POLYGONS_PATH = "../data/city-hex-polygons-8.geojson"
TEST_DATA_PATH = "../data/sr_hex.csv.gz"

# Failure threshold (5%)
FAILURE_THRESHOLD = 0.05

In [ ]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

start_time = time.perf_counter()

Import data

In [ ]:
# Read service requests data
sr = pd.read_csv(INPUT_PATH)
sr_hex = pd.read_csv(TEST_DATA_PATH)

# Read geospatial data
poly8 = gpd.read_file(HEX_POLYGONS_PATH)
poly8 = poly8.rename(columns={"index": "h3_level8_index"})

In [4]:
# Check for uniqueness of notification_number in both datasets
assert sr["notification_number"].is_unique
assert sr_hex["notification_number"].is_unique
assert set(sr["notification_number"]) == set(sr_hex["notification_number"])

The next step performs the geospatial join. To do the spatial join, we use `sjoin_nearest` from GeoPandas. For each service request, the `sjoin_nearest` function joins it to the nearest hexagon, based on the centroid of that hexagon.

This is the most time-consuming step, but should not take longer than 8 minutes, even when benchmarked on older hardware. I did consider using the H3 Hexagonal Grid System along with Numpy for distance calculations to optimise for speed, but ultimately concluded that the potential speed gain was not worth the additional complexity for a dataset of this size.

In [ ]:
df = gpd.GeoDataFrame(
    sr,
    geometry=gpd.points_from_xy(sr["longitude"], sr["latitude"]),
    crs="EPSG:4326"   # WGS84 latitude/longitude
)
df = df.to_crs("EPSG:32734")  # Convert to UTM Zone 34S

# Ensure both GeoDataFrames have the same coordinate reference system
poly8 = poly8.to_crs(df.crs)

# Perform the spatial join
# df = gpd.sjoin(df, poly8[["h3_level8_index", "geometry"]], how="left")
df = gpd.sjoin_nearest(df, poly8[["h3_level8_index", "geometry"]], how="left")
df = df.drop(columns=["index_right"])
# Set `h3_level8_index` to '0' where coordinates are missing
df.loc[df["longitude"].isna() | df["latitude"].isna(), "h3_level8_index"] = '0'

# Record the number of records that failed to join (excluding those with missing coordinates)
num_failed_joins = df["h3_level8_index"].isna().sum()
# Calculate the failure rate (excluding those with missing coordinates)
num_invalid_coords = (df["longitude"].isna() | df["latitude"].isna()).sum()
num_valid_coords = df.shape[0] - num_invalid_coords
failure_rate = num_failed_joins / num_valid_coords
logging.info(f"Total records: {df.shape[0]}")
logging.info(f"Records with missing coordinates: {num_invalid_coords}")
logging.info(f"Failed joins: {num_failed_joins} ({failure_rate*100:.2f}%)")

# Script errors out if failure rate exceeds threshold
if failure_rate > FAILURE_THRESHOLD:
    logging.error("Join failure rate exceeds threshold (%.2f%%)", failure_rate * 100)
    sys.exit(1)

# End timing
elapsed = time.perf_counter() - start_time
logging.info(f"Join completed in {elapsed:.2f} seconds")

   Unnamed: 0  notification_number  reference_number  \
0           0            400583534      9.109492e+09   
1           1            400555043      9.108995e+09   
2           2            400589145      9.109614e+09   
3           3            400538915      9.108601e+09   
4           4            400568554               NaN   

          creation_timestamp       completion_timestamp     directorate  \
0  2020-10-07 06:55:18+02:00  2020-10-08 15:36:35+02:00  URBAN MOBILITY   
1  2020-07-09 16:08:13+02:00  2020-07-14 14:27:01+02:00  URBAN MOBILITY   
2  2020-10-27 10:21:59+02:00  2020-10-28 17:48:15+02:00  URBAN MOBILITY   
3  2020-03-19 06:36:06+02:00  2021-03-29 20:34:19+02:00  URBAN MOBILITY   
4  2020-08-25 09:48:42+02:00  2020-08-31 08:41:13+02:00  URBAN MOBILITY   

                        department            branch  \
0  Roads Infrastructure Management  RIM Area Central   
1  Roads Infrastructure Management     RIM Area East   
2  Roads Infrastructure Management     RIM A

Compare with `sr_hex`:

In [ ]:
# Compare with `sr_hex`
df_test = (
    df[["notification_number", "h3_level8_index", "longitude", "latitude"]]
    .merge(
        sr_hex[["notification_number", "h3_level8_index", "longitude", "latitude"]],
        on="notification_number", how="left", suffixes=('_calc', '_test')
    )
)
df_test["match"] = df_test["h3_level8_index_calc"] == df_test["h3_level8_index_test"]
print("Mismatched records compared to `sr_hex.csv`:")
print(df_test['match'].value_counts())

In [ ]:
df_test[~df_test['match']]

,notification_number,h3_level8_index_calc,longitude_calc,latitude_calc,h3_level8_index_test,longitude_test,latitude_test,match
462434,1015950287,88ad36c605fffff,18.774378,-34.044257,88ad36c629fffff,18.774378,-34.044257,False
804983,1016348681,88ad36c605fffff,18.774378,-34.044257,88ad36c629fffff,18.774378,-34.044257,False
924595,1016488950,88ad361b5bfffff,18.723060,-33.904955,88ad361b51fffff,18.723060,-33.904955,False
